In [2]:
import tensorflow as tf
import keras
import gensim
import numpy as np
import pandas as pd
import logging  
import time  
import os  
import jieba  
import glob  
import random  
import copy  
import chardet  
from gensim import corpora,similarities, models  
from pprint import pprint  
import jieba.analyse
import jieba.posseg as pseg
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.decomposition import PCA  
import sklearn

Using TensorFlow backend.
D:\program\Anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
df_doctor_order = pd.read_csv('4_outpatient_doctor_order_detail.csv')
df_doctor_order.head()

,HOSPITAL_CODE,SERIAL_NO,ITEM_NO,ITEM_CODE,ITEM_NAME,BASE_DOSE,DOSE_UNIT,SPECS,DOSE_ONCE,TOTAL,...,DRUG_IND,COSTS,EXEC_CODE,EXEC_DPNM,UPDATED_ON,SYS_ITEM_NAME,SYS_DOSE_UNIT,SYS_UNIT,SYS_USE_NAME,ITEM_CLASS
0,43120181600,1367431129172,68751496,23357,（急）B型钠尿肽前体（PRO-BNP）测定,NaN,NaN,1,NaN,1.0,...,0,220.0,NaN,NaN,31-7月 -18 04.20.09.000000 下午,NaN,NaN,NaN,NaN,NaN
1,43120181600,1367431129172,68751497,23496,（急）降钙素原检测（定量）,NaN,NaN,1,NaN,1.0,...,0,150.0,NaN,NaN,31-7月 -18 04.20.09.000000 下午,NaN,NaN,NaN,NaN,NaN
2,43120181600,558100129171,61151855,20431,（急）血清肌酸激酶－MB同工酶质量测定,NaN,NaN,1,NaN,1.0,...,0,50.0,422.0,血液细胞室,31-7月 -18 04.20.09.000000 下午,NaN,NaN,NaN,NaN,NaN
3,43120181600,558100129171,61151853,20436,（急）血清尿酸测定,NaN,NaN,1,NaN,1.0,...,0,8.0,422.0,血液细胞室,31-7月 -18 04.20.09.000000 下午,NaN,NaN,NaN,NaN,NaN
4,43120181600,558100129171,61151850,23261,（急）血清总蛋白测定（化学法等）,NaN,NaN,1,NaN,1.0,...,0,5.0,422.0,血液细胞室,31-7月 -18 04.20.09.000000 下午,NaN,NaN,NaN,NaN,NaN


In [4]:
df_doctor_order.shape

(5644, 24)

In [5]:
df_doctor_order.dtypes

HOSPITAL_CODE      object
SERIAL_NO           int64
ITEM_NO             int64
ITEM_CODE          object
ITEM_NAME          object
BASE_DOSE         float64
DOSE_UNIT          object
SPECS              object
DOSE_ONCE         float64
TOTAL             float64
UNIT               object
FREQ_CODE          object
FREQUENCY_NAME     object
USE_NAME           object
DRUG_IND            int64
COSTS             float64
EXEC_CODE         float64
EXEC_DPNM          object
UPDATED_ON         object
SYS_ITEM_NAME      object
SYS_DOSE_UNIT      object
SYS_UNIT           object
SYS_USE_NAME       object
ITEM_CLASS         object
dtype: object

In [6]:
df_doctor_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5644 entries, 0 to 5643
Data columns (total 24 columns):
HOSPITAL_CODE     5644 non-null object
SERIAL_NO         5644 non-null int64
ITEM_NO           5644 non-null int64
ITEM_CODE         5644 non-null object
ITEM_NAME         5644 non-null object
BASE_DOSE         241 non-null float64
DOSE_UNIT         2001 non-null object
SPECS             5608 non-null object
DOSE_ONCE         2011 non-null float64
TOTAL             5644 non-null float64
UNIT              5644 non-null object
FREQ_CODE         2008 non-null object
FREQUENCY_NAME    2007 non-null object
USE_NAME          2001 non-null object
DRUG_IND          5644 non-null int64
COSTS             5644 non-null float64
EXEC_CODE         4650 non-null float64
EXEC_DPNM         4650 non-null object
UPDATED_ON        5644 non-null object
SYS_ITEM_NAME     6 non-null object
SYS_DOSE_UNIT     161 non-null object
SYS_UNIT          98 non-null object
SYS_USE_NAME      122 non-null object
IT

In [7]:
df_doctor_order.describe()

,SERIAL_NO,ITEM_NO,BASE_DOSE,DOSE_ONCE,TOTAL,DRUG_IND,COSTS,EXEC_CODE
count,5.644000e+03,5.644000e+03,241.000000,2011.000000,5644.000000,5644.000000,5644.000000,4650.000000
mean,6.758626e+11,3.765536e+07,20.361286,78.988456,3.732636,0.354536,39.742332,334.604946
std,7.877234e+11,3.231003e+07,55.499145,199.411771,18.298505,0.478415,114.726997,123.161066
min,3.782790e+05,1.000000e+00,0.000000,0.000000,1.000000,0.000000,0.108000,102.000000
25%,1.712015e+10,2.000335e+06,1.000000,3.500000,1.000000,0.000000,4.000000,201.000000
50%,5.183511e+11,6.084686e+07,1.000000,30.000000,1.000000,0.000000,8.000000,413.000000
75%,1.329844e+12,6.841651e+07,8.000000,100.000000,1.000000,1.000000,35.000000,421.000000
max,1.735939e+13,7.442625e+07,400.000000,4250.000000,420.000000,1.000000,3920.000000,1110.000000


In [8]:
for i, v in enumerate(df_doctor_order.columns):
    print(i, v)

0 HOSPITAL_CODE
1 SERIAL_NO
2 ITEM_NO
3 ITEM_CODE
4 ITEM_NAME
5 BASE_DOSE
6 DOSE_UNIT
7 SPECS
8 DOSE_ONCE
9 TOTAL
10 UNIT
11 FREQ_CODE
12 FREQUENCY_NAME
13 USE_NAME
14 DRUG_IND
15 COSTS
16 EXEC_CODE
17 EXEC_DPNM
18 UPDATED_ON
19 SYS_ITEM_NAME
20 SYS_DOSE_UNIT
21 SYS_UNIT
22 SYS_USE_NAME
23 ITEM_CLASS


In [9]:
sum(df_doctor_order.duplicated())

0

In [10]:
df_exam_info = pd.read_csv('9_outpatient_exam_info.csv')
df_exam_info.head()

,HOSPITAL_CODE,CHECKSERIALNUM,INPATIENT_NO,PATIENT_ID,CLINIC_ID,PATIENT_NO,PATIENTNAME,PROVINCE,SEX,AGE,...,OUTP_PATIENT_NO,EXAM_EMP_NO,EXAM_DOCTOR,SEQ_ID,OPEN_SYS_EMP_NO,DATA_VERSION,SYS_CHECKMETHOD,SYS_ITEMNAME,SYS_CHECKNAME,PATI_ID
0,43120181600,20170610000407,NaN,1000073494,2857340.0,NaN,杨*,530102********271X,男,68,...,905405.0,NaN,NaN,1335,NaN,0,NaN,NaN,NaN,NaN
1,43120181600,20170612002668,NaN,5300-0000092951,3804144.0,NaN,章*仁,530102********2112,男,75,...,1200132.0,NaN,NaN,303,NaN,0,NaN,NaN,NaN,NaN
2,43120181600,20170707001233,NaN,1000157523,3385061.0,NaN,余*亚,520202********0017,男,71,...,1069537.0,NaN,NaN,1229,NaN,0,NaN,NaN,NaN,NaN
3,43120181600,20170707001397,NaN,1000157523,3385061.0,NaN,余*亚,520202********0017,男,71,...,1069537.0,NaN,NaN,1230,NaN,0,NaN,NaN,NaN,NaN
4,43120181600,20170710002117,587.0,0101035250,NaN,8000418.0,高*,530102********2431,男,38,...,NaN,NaN,NaN,4186,NaN,0,NaN,NaN,NaN,NaN


In [11]:
df_exam_info.shape

(1795, 46)

In [12]:
df_exam_info.dtypes

HOSPITAL_CODE        int64
CHECKSERIALNUM       int64
INPATIENT_NO       float64
PATIENT_ID          object
CLINIC_ID          float64
PATIENT_NO         float64
PATIENTNAME         object
PROVINCE            object
SEX                 object
AGE                  int64
PATIENT_TYPE        object
BIRTHDAY            object
PHONENUMBER         object
DEVICETYPECODE       int64
DEVICETYPENAME      object
APPLY_NO            object
IMAGEID             object
ITEM_SNOMED        float64
ITEM_CODE           object
ITEMNAME            object
CHECKMETHOD         object
PICTURENUM           int64
EMERGENCY_TREAT    float64
IFMASCULINE          int64
EXEC_DPNMID        float64
EXEC_DPNM           object
OPEN_EMP_NO        float64
OPEN_DOCTOR         object
STUDYTIME          float64
STUDYSTATUSNAME     object
REPORTDESCRIBE      object
REPORTDIAGNOSE      object
REP_EMP_NO         float64
DOCNAME             object
OPERATETIME         object
UPDATED_ON          object
OUTP_PATIENT_NO    float64
E

In [13]:
df_exam_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 46 columns):
HOSPITAL_CODE      1795 non-null int64
CHECKSERIALNUM     1795 non-null int64
INPATIENT_NO       1587 non-null float64
PATIENT_ID         1795 non-null object
CLINIC_ID          208 non-null float64
PATIENT_NO         1587 non-null float64
PATIENTNAME        1795 non-null object
PROVINCE           1795 non-null object
SEX                1795 non-null object
AGE                1795 non-null int64
PATIENT_TYPE       1795 non-null object
BIRTHDAY           1795 non-null object
PHONENUMBER        1795 non-null object
DEVICETYPECODE     1795 non-null int64
DEVICETYPENAME     1795 non-null object
APPLY_NO           1791 non-null object
IMAGEID            1795 non-null object
ITEM_SNOMED        0 non-null float64
ITEM_CODE          1795 non-null object
ITEMNAME           1795 non-null object
CHECKMETHOD        1795 non-null object
PICTURENUM         1795 non-null int64
EMERGENCY_TREAT   

In [14]:
for i, v in enumerate(df_exam_info.columns):
    print(i, v)

0 HOSPITAL_CODE
1 CHECKSERIALNUM
2 INPATIENT_NO
3 PATIENT_ID
4 CLINIC_ID
5 PATIENT_NO
6 PATIENTNAME
7 PROVINCE
8 SEX
9 AGE
10 PATIENT_TYPE
11 BIRTHDAY
12 PHONENUMBER
13 DEVICETYPECODE
14 DEVICETYPENAME
15 APPLY_NO
16 IMAGEID
17 ITEM_SNOMED
18 ITEM_CODE
19 ITEMNAME
20 CHECKMETHOD
21 PICTURENUM
22 EMERGENCY_TREAT
23 IFMASCULINE
24 EXEC_DPNMID
25 EXEC_DPNM
26 OPEN_EMP_NO
27 OPEN_DOCTOR
28 STUDYTIME
29 STUDYSTATUSNAME
30 REPORTDESCRIBE
31 REPORTDIAGNOSE
32 REP_EMP_NO
33 DOCNAME
34 OPERATETIME
35 UPDATED_ON
36 OUTP_PATIENT_NO
37 EXAM_EMP_NO
38 EXAM_DOCTOR
39 SEQ_ID
40 OPEN_SYS_EMP_NO
41 DATA_VERSION
42 SYS_CHECKMETHOD
43 SYS_ITEMNAME
44 SYS_CHECKNAME
45 PATI_ID


In [15]:
sum(df_exam_info.duplicated())

0

In [16]:
df_main_doctorOrder = pd.read_csv('3_outpatient_doctor_order_main_record.csv')
df_main_doctorOrder.head()

,HOSPITAL_CODE,SERIAL_NO,VISIT_DATE,OUTP_PATIENT_NO,DEPT_CODE,ORDERED_BY,EMP_NO,DOCTOR,ORDERED_DATE,UPDATED_ON,SEQ_ID,OPEN_SYS_EMP_NO,DATA_VERSION
0,43120181600,1216818129171,2018/1/16,1216818,18.0,急诊内科,30380,杨爱华,2018/1/16,31-7月 -18 04.13.08.000000 下午,2312287,NaN,0
1,43120181600,1216818129221,2018/1/16,1216818,18.0,急诊内科,30380,杨爱华,2018/1/16,31-7月 -18 04.13.08.000000 下午,2313645,NaN,0
2,43120181600,1216818129281,2018/1/16,1216818,18.0,急诊内科,30380,杨爱华,2018/1/16,31-7月 -18 04.13.08.000000 下午,2313203,NaN,0
3,43120181600,18011654389,2018/1/16,1216818,18.0,急诊内科,30380,杨爱华,2018/1/16,31-7月 -18 04.13.08.000000 下午,2311209,NaN,0
4,43120181600,1438773129172,2018/3/1,1438773,18.0,急诊内科,30391,曹云友,2018/3/1,31-7月 -18 04.13.08.000000 下午,2313743,NaN,0


In [17]:
df_main_doctorOrder.shape

(1204, 13)

In [18]:
df_main_doctorOrder.dtypes

HOSPITAL_CODE       object
SERIAL_NO            int64
VISIT_DATE          object
OUTP_PATIENT_NO      int64
DEPT_CODE          float64
ORDERED_BY          object
EMP_NO               int64
DOCTOR              object
ORDERED_DATE        object
UPDATED_ON          object
SEQ_ID               int64
OPEN_SYS_EMP_NO    float64
DATA_VERSION         int64
dtype: object

In [19]:
df_main_doctorOrder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204 entries, 0 to 1203
Data columns (total 13 columns):
HOSPITAL_CODE      1204 non-null object
SERIAL_NO          1204 non-null int64
VISIT_DATE         1204 non-null object
OUTP_PATIENT_NO    1204 non-null int64
DEPT_CODE          1202 non-null float64
ORDERED_BY         1204 non-null object
EMP_NO             1204 non-null int64
DOCTOR             1204 non-null object
ORDERED_DATE       1204 non-null object
UPDATED_ON         1204 non-null object
SEQ_ID             1204 non-null int64
OPEN_SYS_EMP_NO    0 non-null float64
DATA_VERSION       1204 non-null int64
dtypes: float64(2), int64(5), object(6)
memory usage: 122.4+ KB


In [20]:
for i, v in enumerate(df_main_doctorOrder.columns):
    print(i, v)

0 HOSPITAL_CODE
1 SERIAL_NO
2 VISIT_DATE
3 OUTP_PATIENT_NO
4 DEPT_CODE
5 ORDERED_BY
6 EMP_NO
7 DOCTOR
8 ORDERED_DATE
9 UPDATED_ON
10 SEQ_ID
11 OPEN_SYS_EMP_NO
12 DATA_VERSION


In [21]:
sum(df_main_doctorOrder.duplicated())

0

2.下面从各数据表中提取深度学习训练用到的关键数据，并通过外键进行join连接

In [22]:
df_exam_exact = df_exam_info.loc[:, ['OUTP_PATIENT_NO','REPORTDESCRIBE','REPORTDIAGNOSE']]
df_exam_exact.head()

,OUTP_PATIENT_NO,REPORTDESCRIBE,REPORTDIAGNOSE
0,905405.0,2017年6月10日16:21:08\n\n急诊报告：\n\n1.慢性支气管炎CT征象。\n...,1.慢性支气管炎CT征象。\n2.双肺见小结节、条索影，多考虑为慢性感染性病变，建议治疗后复...
1,1200132.0,CDFI及CDE彩色多普勒超声宽景成像显示：\n双侧下肢（股总、股浅、股深近心段、腘、胫后、...,1.左侧下肢股总静脉瓣膜功能不全（轻度）。\n2.左侧下肢小腿段浅静脉曲张。\n3.双侧下肢...
2,1069537.0,2017年7月7日10:20:19\n\n 腰椎生理曲度变直，序列整齐，椎体边缘变尖，...,1、腰椎退行性改变。\n2、腰5-骶1关节间隙变窄。
3,1069537.0,\n 腰椎生理曲度变直，各腰椎椎体骨质前后缘增生变尖，尤以腰5椎体...,各腰椎椎体骨质前后缘增生变尖，尤以腰5椎体明显，腰5椎体下缘及骶1椎体上缘异常信号影，...
4,NaN,NaN,NaN


In [23]:
df_exam_exact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 3 columns):
OUTP_PATIENT_NO    208 non-null float64
REPORTDESCRIBE     1615 non-null object
REPORTDIAGNOSE     1611 non-null object
dtypes: float64(1), object(2)
memory usage: 42.1+ KB


In [24]:
for i in range(len(df_exam_exact)):
  if np.isnan(df_exam_exact['OUTP_PATIENT_NO'][i]) or pd.isnull(df_exam_exact['REPORTDIAGNOSE'][i]):
    df_exam_exact = df_exam_exact.drop(i)

df_exam_exact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 1773
Data columns (total 3 columns):
OUTP_PATIENT_NO    199 non-null float64
REPORTDESCRIBE     199 non-null object
REPORTDIAGNOSE     199 non-null object
dtypes: float64(1), object(2)
memory usage: 6.2+ KB


In [25]:
df_exam_exact['wordlist'] = df_exam_exact['REPORTDESCRIBE'].map(lambda x: list(jieba.cut(x)))
# df_exam_exact['wordlist'] = df_exam_exact['REPORTDESCRIBE'].map(lambda x: list(x))

df_exam_exact.head()

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Acer\AppData\Local\Temp\jieba.cache
Loading model cost 1.041 seconds.
Prefix dict has been built succesfully.


,OUTP_PATIENT_NO,REPORTDESCRIBE,REPORTDIAGNOSE,wordlist
0,905405.0,2017年6月10日16:21:08\n\n急诊报告：\n\n1.慢性支气管炎CT征象。\n...,1.慢性支气管炎CT征象。\n2.双肺见小结节、条索影，多考虑为慢性感染性病变，建议治疗后复...,"[2017, 年, 6, 月, 10, 日, 16, :, 21, :, 08, \n, \..."
1,1200132.0,CDFI及CDE彩色多普勒超声宽景成像显示：\n双侧下肢（股总、股浅、股深近心段、腘、胫后、...,1.左侧下肢股总静脉瓣膜功能不全（轻度）。\n2.左侧下肢小腿段浅静脉曲张。\n3.双侧下肢...,"[CDFI, 及, CDE, 彩色, 多普勒, 超声, 宽景, 成像, 显示, ：, \n,..."
2,1069537.0,2017年7月7日10:20:19\n\n 腰椎生理曲度变直，序列整齐，椎体边缘变尖，...,1、腰椎退行性改变。\n2、腰5-骶1关节间隙变窄。,"[2017, 年, 7, 月, 7, 日, 10, :, 20, :, 19, \n, \n..."
3,1069537.0,\n 腰椎生理曲度变直，各腰椎椎体骨质前后缘增生变尖，尤以腰5椎体...,各腰椎椎体骨质前后缘增生变尖，尤以腰5椎体明显，腰5椎体下缘及骶1椎体上缘异常信号影，...,"[ , , , , , , , , , , \n, , , , ,..."
8,483018.0,急诊报告\n2017年8月15日15:50:46\n1、双侧基底节区、脑干区点状低密度影，考...,1、双侧基底节区、脑干区点状低密度影，考虑为腔隙性脑梗塞。\n2、老年性脑改变。\n3、CT...,"[急诊, 报告, \n, 2017, 年, 8, 月, 15, 日, 15, :, 50, ..."


In [26]:
df_exam_exact['keywords'] = df_exam_exact['REPORTDESCRIBE'].map(lambda x:jieba.analyse.extract_tags(x,10))
df_exam_exact.head()

,OUTP_PATIENT_NO,REPORTDESCRIBE,REPORTDIAGNOSE,wordlist,keywords
0,905405.0,2017年6月10日16:21:08\n\n急诊报告：\n\n1.慢性支气管炎CT征象。\n...,1.慢性支气管炎CT征象。\n2.双肺见小结节、条索影，多考虑为慢性感染性病变，建议治疗后复...,"[2017, 年, 6, 月, 10, 日, 16, :, 21, :, 08, \n, \...","[征象, 肝脏, 胆囊结石, 慢性感染, 2017, 10, 16, 21, 08, CT]"
1,1200132.0,CDFI及CDE彩色多普勒超声宽景成像显示：\n双侧下肢（股总、股浅、股深近心段、腘、胫后、...,1.左侧下肢股总静脉瓣膜功能不全（轻度）。\n2.左侧下肢小腿段浅静脉曲张。\n3.双侧下肢...,"[CDFI, 及, CDE, 彩色, 多普勒, 超声, 宽景, 成像, 显示, ：, \n,...","[未见异常, CDFI, CDE, 血流, 下肢, 示管腔, 内径, 小腿, 静脉, 信号]"
2,1069537.0,2017年7月7日10:20:19\n\n 腰椎生理曲度变直，序列整齐，椎体边缘变尖，...,1、腰椎退行性改变。\n2、腰5-骶1关节间隙变窄。,"[2017, 年, 7, 月, 7, 日, 10, :, 20, :, 19, \n, \n...","[曲度, 2017, 10, 20, 19, 变直, 变尖, 骨赘, 椎旁, 内未见]"
3,1069537.0,\n 腰椎生理曲度变直，各腰椎椎体骨质前后缘增生变尖，尤以腰5椎体...,各腰椎椎体骨质前后缘增生变尖，尤以腰5椎体明显，腰5椎体下缘及骶1椎体上缘异常信号影，...,"[ , , , , , , , , , , \n, , , , ,...","[椎体, 腰椎, 椎间盘, 信号, 神经, 终丝, 未见异常, 曲度, 变直, 变尖]"
8,483018.0,急诊报告\n2017年8月15日15:50:46\n1、双侧基底节区、脑干区点状低密度影，考...,1、双侧基底节区、脑干区点状低密度影，考虑为腔隙性脑梗塞。\n2、老年性脑改变。\n3、CT...,"[急诊, 报告, \n, 2017, 年, 8, 月, 15, 日, 15, :, 50, ...","[15, 基底节, 2017, 50, 46, 双侧, 为腔隙, 脑梗塞, CT, 平扫脑]"


In [27]:
df_exam_exact.reset_index(drop=True)

,OUTP_PATIENT_NO,REPORTDESCRIBE,REPORTDIAGNOSE,wordlist,keywords
0,905405.0,2017年6月10日16:21:08\n\n急诊报告：\n\n1.慢性支气管炎CT征象。\n...,1.慢性支气管炎CT征象。\n2.双肺见小结节、条索影，多考虑为慢性感染性病变，建议治疗后复...,"[2017, 年, 6, 月, 10, 日, 16, :, 21, :, 08, \n, \...","[征象, 肝脏, 胆囊结石, 慢性感染, 2017, 10, 16, 21, 08, CT]"
1,1200132.0,CDFI及CDE彩色多普勒超声宽景成像显示：\n双侧下肢（股总、股浅、股深近心段、腘、胫后、...,1.左侧下肢股总静脉瓣膜功能不全（轻度）。\n2.左侧下肢小腿段浅静脉曲张。\n3.双侧下肢...,"[CDFI, 及, CDE, 彩色, 多普勒, 超声, 宽景, 成像, 显示, ：, \n,...","[未见异常, CDFI, CDE, 血流, 下肢, 示管腔, 内径, 小腿, 静脉, 信号]"
2,1069537.0,2017年7月7日10:20:19\n\n 腰椎生理曲度变直，序列整齐，椎体边缘变尖，...,1、腰椎退行性改变。\n2、腰5-骶1关节间隙变窄。,"[2017, 年, 7, 月, 7, 日, 10, :, 20, :, 19, \n, \n...","[曲度, 2017, 10, 20, 19, 变直, 变尖, 骨赘, 椎旁, 内未见]"
3,1069537.0,\n 腰椎生理曲度变直，各腰椎椎体骨质前后缘增生变尖，尤以腰5椎体...,各腰椎椎体骨质前后缘增生变尖，尤以腰5椎体明显，腰5椎体下缘及骶1椎体上缘异常信号影，...,"[ , , , , , , , , , , \n, , , , ,...","[椎体, 腰椎, 椎间盘, 信号, 神经, 终丝, 未见异常, 曲度, 变直, 变尖]"
4,483018.0,急诊报告\n2017年8月15日15:50:46\n1、双侧基底节区、脑干区点状低密度影，考...,1、双侧基底节区、脑干区点状低密度影，考虑为腔隙性脑梗塞。\n2、老年性脑改变。\n3、CT...,"[急诊, 报告, \n, 2017, 年, 8, 月, 15, 日, 15, :, 50, ...","[15, 基底节, 2017, 50, 46, 双侧, 为腔隙, 脑梗塞, CT, 平扫脑]"
5,409793.0,急诊报告\n2017年8月17日16:20:38\n1、左侧少量气胸，左肺组织被压缩约5%。...,1、左侧少量气胸，左肺组织被压缩约5%。\n2、双肺间质纤维化改变，右肺上、中叶、双肺下叶支...,"[急诊, 报告, \n, 2017, 年, 8, 月, 17, 日, 16, :, 20, ...","[双肺, 双侧, 少量, 胸膜, 积液, 右肺, 2017, 17, 16, 20]"
6,757686.0,2017年8月18日09:35:57\n\n 定位像示颈椎生理曲度存在，各椎间隙无变窄...,1、颈椎退行性改变，颈2-3，颈3-4椎间盘膨出，颈4-5椎间盘突出，颈4-6椎体层面硬膜囊...,"[2017, 年, 8, 月, 18, 日, 09, :, 35, :, 57, \n, \...","[椎体, 变窄, 椎间隙, 椎间盘, 后缘, 层面, 顺位, 曲度, 2017, 18]"
7,409853.0,2017年8月17日22:26:27\n急诊报告；\n1.慢性支气管炎、肺气肿、肺心病CT征...,1.慢性支气管炎、肺气肿、肺心病CT征象。\n2.右肺上叶尖段、中叶及左肺下叶外基底段见小结...,"[2017, 年, 8, 月, 17, 日, 22, :, 26, :, 27, \n, 急...","[双侧, 右肺, 胆囊结石, 胸内, 上叶, 2017, 17, 22, 26, 27]"
8,430115.0,2017年8月23日9:01:59\n 胸廓对称，纵隔气管未见移位。双肺支气管血管束增...,1、慢性支气管炎、肺气肿，左肺下叶支气管扩张合并感染。\n2、冠状动脉及胸主动脉壁钙化。,"[2017, 年, 8, 月, 23, 日, 9, :, 01, :, 59, \n, ,...","[双肺, 支气管, 未见, 2017, 23, 01, 59, 增粗, 左肺, 双侧]"
9,442894.0,2017年8月24日8:51:58\n 胸廓对称，纵隔气管未见移位。双肺纹理增多、增粗...,1、慢性支气管炎、肺气肿征并多发肺大泡形成。\n2、双侧胸膜局限性轻度增厚、粘连。\n3、心...,"[2017, 年, 8, 月, 24, 日, 8, :, 51, :, 58, \n, ,...","[未见, 双肺, 双侧, 纵隔, 征象, 2017, 24, 51, 58, 增粗]"


2.2 接下来提取医嘱主记录表和医嘱明细表的关键数据



In [28]:
df_main_doctorOrder_exact = df_main_doctorOrder[['SERIAL_NO','OUTP_PATIENT_NO']]
df_main_doctorOrder_exact.head()

,SERIAL_NO,OUTP_PATIENT_NO
0,1216818129171,1216818
1,1216818129221,1216818
2,1216818129281,1216818
3,18011654389,1216818
4,1438773129172,1438773


In [29]:
df_main_doctorOrder_exact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204 entries, 0 to 1203
Data columns (total 2 columns):
SERIAL_NO          1204 non-null int64
OUTP_PATIENT_NO    1204 non-null int64
dtypes: int64(2)
memory usage: 18.9 KB


In [30]:
df_doctorOrder_detail_exact = df_doctor_order[['SERIAL_NO','ITEM_CODE','ITEM_NAME']]
df_doctorOrder_detail_exact.head()

,SERIAL_NO,ITEM_CODE,ITEM_NAME
0,1367431129172,23357,（急）B型钠尿肽前体（PRO-BNP）测定
1,1367431129172,23496,（急）降钙素原检测（定量）
2,558100129171,20431,（急）血清肌酸激酶－MB同工酶质量测定
3,558100129171,20436,（急）血清尿酸测定
4,558100129171,23261,（急）血清总蛋白测定（化学法等）


In [31]:
df_doctorOrder_detail_exact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5644 entries, 0 to 5643
Data columns (total 3 columns):
SERIAL_NO    5644 non-null int64
ITEM_CODE    5644 non-null object
ITEM_NAME    5644 non-null object
dtypes: int64(1), object(2)
memory usage: 132.4+ KB


2.4 接下来进行聚合操作



In [32]:
df_join_exam_and_mainDoctorOrder_exact = pd.merge(df_exam_exact, df_main_doctorOrder_exact, how='inner', on=['OUTP_PATIENT_NO'])
df_join_exam_and_mainDoctorOrder_exact.head()

,OUTP_PATIENT_NO,REPORTDESCRIBE,REPORTDIAGNOSE,wordlist,keywords,SERIAL_NO
0,483018.0,急诊报告\n2017年8月15日15:50:46\n1、双侧基底节区、脑干区点状低密度影，考...,1、双侧基底节区、脑干区点状低密度影，考虑为腔隙性脑梗塞。\n2、老年性脑改变。\n3、CT...,"[急诊, 报告, \n, 2017, 年, 8, 月, 15, 日, 15, :, 50, ...","[15, 基底节, 2017, 50, 46, 双侧, 为腔隙, 脑梗塞, CT, 平扫脑]",17083053661
1,483018.0,急诊报告\n2017年8月15日15:50:46\n1、双侧基底节区、脑干区点状低密度影，考...,1、双侧基底节区、脑干区点状低密度影，考虑为腔隙性脑梗塞。\n2、老年性脑改变。\n3、CT...,"[急诊, 报告, \n, 2017, 年, 8, 月, 15, 日, 15, :, 50, ...","[15, 基底节, 2017, 50, 46, 双侧, 为腔隙, 脑梗塞, CT, 平扫脑]",17083053669
2,483018.0,急诊报告\n2017年8月15日15:50:46\n1、双侧基底节区、脑干区点状低密度影，考...,1、双侧基底节区、脑干区点状低密度影，考虑为腔隙性脑梗塞。\n2、老年性脑改变。\n3、CT...,"[急诊, 报告, \n, 2017, 年, 8, 月, 15, 日, 15, :, 50, ...","[15, 基底节, 2017, 50, 46, 双侧, 为腔隙, 脑梗塞, CT, 平扫脑]",448637
3,483018.0,急诊报告\n2017年8月15日15:50:46\n1、双侧基底节区、脑干区点状低密度影，考...,1、双侧基底节区、脑干区点状低密度影，考虑为腔隙性脑梗塞。\n2、老年性脑改变。\n3、CT...,"[急诊, 报告, \n, 2017, 年, 8, 月, 15, 日, 15, :, 50, ...","[15, 基底节, 2017, 50, 46, 双侧, 为腔隙, 脑梗塞, CT, 平扫脑]",448638
4,483018.0,急诊报告\n2017年8月15日15:50:46\n1、双侧基底节区、脑干区点状低密度影，考...,1、双侧基底节区、脑干区点状低密度影，考虑为腔隙性脑梗塞。\n2、老年性脑改变。\n3、CT...,"[急诊, 报告, \n, 2017, 年, 8, 月, 15, 日, 15, :, 50, ...","[15, 基底节, 2017, 50, 46, 双侧, 为腔隙, 脑梗塞, CT, 平扫脑]",483018129174


In [33]:
df_join_exam_and_mainDoctorOrder_exact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 717 entries, 0 to 716
Data columns (total 6 columns):
OUTP_PATIENT_NO    717 non-null float64
REPORTDESCRIBE     717 non-null object
REPORTDIAGNOSE     717 non-null object
wordlist           717 non-null object
keywords           717 non-null object
SERIAL_NO          717 non-null int64
dtypes: float64(1), int64(1), object(4)
memory usage: 39.2+ KB


In [34]:
df_join_exam_and_doctorOrderDetail_exact = pd.merge(df_join_exam_and_mainDoctorOrder_exact, df_doctorOrder_detail_exact, how='inner', on=['SERIAL_NO'])
df_join_exam_and_doctorOrderDetail_exact.head()

,OUTP_PATIENT_NO,REPORTDESCRIBE,REPORTDIAGNOSE,wordlist,keywords,SERIAL_NO,ITEM_CODE,ITEM_NAME
0,483018.0,急诊报告\n2017年8月15日15:50:46\n1、双侧基底节区、脑干区点状低密度影，考...,1、双侧基底节区、脑干区点状低密度影，考虑为腔隙性脑梗塞。\n2、老年性脑改变。\n3、CT...,"[急诊, 报告, \n, 2017, 年, 8, 月, 15, 日, 15, :, 50, ...","[15, 基底节, 2017, 50, 46, 双侧, 为腔隙, 脑梗塞, CT, 平扫脑]",17083053661,2434,头孢甲肟
1,483018.0,2017年8月30日20:06:15\n急诊报告：\n1、慢性支气管炎、肺气肿征象。\n2、...,1、慢性支气管炎、肺气肿征象。\n2、左肺上叶尖后段局部支气管扩张。\n3、双肺上叶尖（后）...,"[2017, 年, 8, 月, 30, 日, 20, :, 06, :, 15, \n, 急...","[左肺, 叶尖, 征象, 叶斑, 2017, 30, 20, 06, 15, 双肺]",17083053661,2434,头孢甲肟
2,483018.0,急诊报告\n2017年8月15日15:50:46\n1、双侧基底节区、脑干区点状低密度影，考...,1、双侧基底节区、脑干区点状低密度影，考虑为腔隙性脑梗塞。\n2、老年性脑改变。\n3、CT...,"[急诊, 报告, \n, 2017, 年, 8, 月, 15, 日, 15, :, 50, ...","[15, 基底节, 2017, 50, 46, 双侧, 为腔隙, 脑梗塞, CT, 平扫脑]",17083053669,101032,柴胡注射液(神威)
3,483018.0,急诊报告\n2017年8月15日15:50:46\n1、双侧基底节区、脑干区点状低密度影，考...,1、双侧基底节区、脑干区点状低密度影，考虑为腔隙性脑梗塞。\n2、老年性脑改变。\n3、CT...,"[急诊, 报告, \n, 2017, 年, 8, 月, 15, 日, 15, :, 50, ...","[15, 基底节, 2017, 50, 46, 双侧, 为腔隙, 脑梗塞, CT, 平扫脑]",17083053669,1623,0.9%氯化钠
4,483018.0,急诊报告\n2017年8月15日15:50:46\n1、双侧基底节区、脑干区点状低密度影，考...,1、双侧基底节区、脑干区点状低密度影，考虑为腔隙性脑梗塞。\n2、老年性脑改变。\n3、CT...,"[急诊, 报告, \n, 2017, 年, 8, 月, 15, 日, 15, :, 50, ...","[15, 基底节, 2017, 50, 46, 双侧, 为腔隙, 脑梗塞, CT, 平扫脑]",17083053669,1645,5%葡萄糖


In [35]:
df_join_exam_and_doctorOrderDetail_exact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3601 entries, 0 to 3600
Data columns (total 8 columns):
OUTP_PATIENT_NO    3601 non-null float64
REPORTDESCRIBE     3601 non-null object
REPORTDIAGNOSE     3601 non-null object
wordlist           3601 non-null object
keywords           3601 non-null object
SERIAL_NO          3601 non-null int64
ITEM_CODE          3601 non-null object
ITEM_NAME          3601 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 253.2+ KB


# 3. 接下来进行对诊断信息分词进行word embedding

In [36]:
diagnose_words_lists = df_join_exam_and_doctorOrderDetail_exact['wordlist'].values
diagnose_all_words = np.hstack(diagnose_words_lists)
diagnose_word_bag = set(diagnose_all_words)
diagnose_word_bag

{'马尾',
 '大小',
 '脑梗死',
 '后',
 '结石',
 '背阔',
 'PW',
 '蝶骨',
 '消失',
 '右肾',
 '2.4',
 '筋膜',
 '欠均',
 '破坏',
 '呈',
 '终丝',
 '性',
 '置入',
 '排外',
 '旁及',
 '4.4',
 '走行区',
 '/',
 '脑梗塞',
 '颈内',
 '角未见',
 '面',
 '肾盂',
 '包裹',
 '颅底',
 '1.81',
 '弥漫',
 '连续',
 '变窄',
 '片影',
 '偏移',
 '脑干',
 '毛糙',
 '透',
 '胸腹',
 '0.3',
 '脊柱',
 '囊',
 '炎症',
 '分别',
 '内散',
 '上',
 '3.8',
 '0.50',
 '粘膜',
 '示双',
 '状强',
 '职业',
 '15',
 '2.0',
 '强',
 '病变',
 '肠管',
 '扫描',
 '子宫',
 '关节',
 '立体',
 '邻近',
 '网格',
 '不张',
 '顶部',
 '近段',
 '三维',
 '胸腔',
 '形成',
 '疾病',
 '01',
 '3mm',
 '位置',
 '增宽',
 '系膜间',
 '腋窝',
 '髁',
 '容积',
 '外踝',
 '叶尖',
 '31',
 '所示',
 '肾',
 '腹直',
 '12mm',
 '肿块',
 '序列',
 '41',
 '亦',
 '丘脑',
 '块状',
 '两个',
 '放射',
 '高密度',
 '门影',
 '纤维',
 '颞',
 '9mm',
 '腰方',
 '08',
 '左叶',
 '45mm',
 '1.1',
 '一',
 'T2WI',
 '团块',
 '回声',
 '锐利',
 '旁',
 '系膜',
 '不佳',
 '肝',
 '圆形',
 '腹股沟',
 '肺',
 '纤维化',
 '基底',
 '于',
 '膈',
 '坐骨',
 '海马',
 '蝶窦',
 '37mm',
 '结合',
 '07',
 '球部',
 '扩大',
 '前',
 '；',
 '树清',
 '中心区',
 '2017',
 '肺叶',
 '无',
 '食管',
 '一囊性',
 '缺如',
 '双上',
 '弧形',
 '高压',
 '稍

In [37]:
len(diagnose_word_bag)

1137

In [38]:
from collections import Counter
counts = Counter(diagnose_word_bag)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word:ii for ii, word in enumerate(vocab, 1)}

diagnoses_ints = []
for each in diagnose_words_lists:
    diagnoses_ints.append([vocab_to_int[word] for word in each])

In [39]:
vocab_to_int

{'马尾': 1,
 '大小': 2,
 '脑梗死': 3,
 '后': 4,
 '结石': 5,
 '背阔': 6,
 'PW': 7,
 '蝶骨': 8,
 '消失': 9,
 '右肾': 10,
 '2.4': 11,
 '筋膜': 12,
 '欠均': 13,
 '破坏': 14,
 '呈': 15,
 '终丝': 16,
 '性': 17,
 '置入': 18,
 '排外': 19,
 '旁及': 20,
 '4.4': 21,
 '走行区': 22,
 '/': 23,
 '脑梗塞': 24,
 '颈内': 25,
 '角未见': 26,
 '面': 27,
 '肾盂': 28,
 '包裹': 29,
 '颅底': 30,
 '1.81': 31,
 '弥漫': 32,
 '连续': 33,
 '变窄': 34,
 '片影': 35,
 '偏移': 36,
 '脑干': 37,
 '毛糙': 38,
 '透': 39,
 '胸腹': 40,
 '0.3': 41,
 '脊柱': 42,
 '囊': 43,
 '炎症': 44,
 '分别': 45,
 '内散': 46,
 '上': 47,
 '3.8': 48,
 '0.50': 49,
 '粘膜': 50,
 '示双': 51,
 '状强': 52,
 '职业': 53,
 '15': 54,
 '2.0': 55,
 '强': 56,
 '病变': 57,
 '肠管': 58,
 '扫描': 59,
 '子宫': 60,
 '关节': 61,
 '立体': 62,
 '邻近': 63,
 '网格': 64,
 '不张': 65,
 '顶部': 66,
 '近段': 67,
 '三维': 68,
 '胸腔': 69,
 '形成': 70,
 '疾病': 71,
 '01': 72,
 '3mm': 73,
 '位置': 74,
 '增宽': 75,
 '系膜间': 76,
 '腋窝': 77,
 '髁': 78,
 '容积': 79,
 '外踝': 80,
 '叶尖': 81,
 '31': 82,
 '所示': 83,
 '肾': 84,
 '腹直': 85,
 '12mm': 86,
 '肿块': 87,
 '序列': 88,
 '41': 89,
 '亦': 90,
 '丘脑': 91,
 '块

In [40]:
diagnoses_ints

[[1064,
  425,
  609,
  133,
  837,
  1115,
  370,
  54,
  1002,
  54,
  861,
  506,
  861,
  397,
  609,
  901,
  659,
  1079,
  418,
  862,
  659,
  37,
  862,
  567,
  369,
  367,
  603,
  486,
  618,
  17,
  24,
  261,
  609,
  221,
  659,
  438,
  1036,
  985,
  261,
  609,
  302,
  659,
  917,
  264,
  156,
  803,
  632,
  179,
  261],
 [133,
  837,
  1115,
  370,
  978,
  1002,
  800,
  861,
  414,
  861,
  54,
  609,
  1064,
  425,
  492,
  609,
  901,
  659,
  316,
  487,
  659,
  839,
  505,
  261,
  609,
  221,
  659,
  694,
  47,
  81,
  4,
  1016,
  727,
  181,
  534,
  261,
  609,
  302,
  659,
  984,
  47,
  81,
  460,
  4,
  349,
  1016,
  518,
  261,
  609,
  1082,
  659,
  694,
  47,
  488,
  756,
  630,
  871,
  367,
  603,
  486,
  263,
  57,
  603,
  1133,
  457,
  4,
  165,
  261,
  609,
  674,
  659,
  888,
  723,
  603,
  1136,
  733,
  153,
  154,
  337,
  675,
  261,
  609,
  929,
  659,
  1079,
  222,
  868,
  659,
  740,
  261,
  609,
  473,
  659,
  59,
  3

In [50]:
diagnose_lens = Counter([len(diagnose) for diagnose in diagnoses_ints])
max(diagnose_lens)

391

In [54]:
seq_len = max(diagnose_lens) 
features = np.zeros((len(diagnoses_ints), seq_len), dtype=int)
for ii, diagnose in enumerate(diagnoses_ints):
    features[ii][-len(diagnose):] = np.array(diagnose)[:seq_len]
    
features[:10][:200]

array([[  0,   0,   0, ..., 632, 179, 261],
       [  0,   0,   0, ..., 182, 505, 261],
       [  0,   0,   0, ..., 632, 179, 261],
       ...,
       [  0,   0,   0, ..., 182, 505, 261],
       [  0,   0,   0, ..., 182, 505, 261],
       [  0,   0,   0, ..., 182, 505, 261]])

In [59]:
doctor_orders = df_join_exam_and_doctorOrderDetail_exact['ITEM_CODE'].values
doctor_orders

array(['2434', '2434', '101032', ..., '23005', '23005', '23895'],
      dtype=object)

In [60]:
doctor_orders_counter = Counter(doctor_orders)
doctor_orders_counter

Counter({'2434': 19,
         '101032': 4,
         '1623': 154,
         '1645': 42,
         '2405': 6,
         '23895': 43,
         '24130': 39,
         '23005': 123,
         '20436': 64,
         '23261': 64,
         '23264': 64,
         '23282': 70,
         '23286': 27,
         '23294': 71,
         '23296': 71,
         '23298': 71,
         '23300': 71,
         '23302': 27,
         '23304': 27,
         '23310': 71,
         '23314': 64,
         '23316': 64,
         '23318': 64,
         '23324': 64,
         '23326': 64,
         '23361': 64,
         '23364': 64,
         '23253': 48,
         '20238': 66,
         '20259': 27,
         '20270': 27,
         '102723': 30,
         '1540': 19,
         '1564': 3,
         '1565': 8,
         '1615': 75,
         '1618': 19,
         '1969': 13,
         '2459': 18,
         '728': 4,
         '831': 18,
         '1620': 10,
         '2299': 6,
         '701': 3,
         '20431': 53,
         '23348': 53,
         '

In [64]:
doctorOrders_to_int = {code:ii for ii,code in enumerate(doctor_orders_counter,1)}
doctorOrders_to_int

{'2434': 1,
 '101032': 2,
 '1623': 3,
 '1645': 4,
 '2405': 5,
 '23895': 6,
 '24130': 7,
 '23005': 8,
 '20436': 9,
 '23261': 10,
 '23264': 11,
 '23282': 12,
 '23286': 13,
 '23294': 14,
 '23296': 15,
 '23298': 16,
 '23300': 17,
 '23302': 18,
 '23304': 19,
 '23310': 20,
 '23314': 21,
 '23316': 22,
 '23318': 23,
 '23324': 24,
 '23326': 25,
 '23361': 26,
 '23364': 27,
 '23253': 28,
 '20238': 29,
 '20259': 30,
 '20270': 31,
 '102723': 32,
 '1540': 33,
 '1564': 34,
 '1565': 35,
 '1615': 36,
 '1618': 37,
 '1969': 38,
 '2459': 39,
 '728': 40,
 '831': 41,
 '1620': 42,
 '2299': 43,
 '701': 44,
 '20431': 45,
 '23348': 46,
 '23350': 47,
 '23496': 48,
 '25120': 49,
 '23214': 50,
 '23224': 51,
 '23228': 52,
 '23236': 53,
 '23246': 54,
 '23250': 55,
 '1079': 56,
 '2100': 57,
 '657': 58,
 '2007': 59,
 '537': 60,
 '20981': 61,
 '20988': 62,
 '20992': 63,
 '20402': 64,
 '20403': 65,
 '23290': 66,
 '23292': 67,
 'Y00000018210': 68,
 '20139': 69,
 '20141': 70,
 '20150': 71,
 '23066': 72,
 '23068': 73,
 '23

In [66]:
doctorOrders_ints = [doctorOrders_to_int[x] for x in doctor_orders]
len(doctorOrders_ints)

3601

# 3.2 训练、验证、测试数据集分割

In [68]:
split_frac = 0.8
split_idx = int(len(features)*split_frac)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = doctorOrders_ints[:split_idx], doctorOrders_ints[split_idx:]

test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(2880, 391) 
Validation set: 	(360, 391) 
Test set: 		(361, 391)


In [69]:
lstm_size = 256
lstm_layer = 1
batch_size = 50
learning_rate = 0.001

In [70]:
n_words = len(vocab_to_int)+1

graph = tf.Graph()
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [ ]:
embed_size = 300